--------------------------
ERP for all raws, left n right
-------------------------

In [ ]:
from eeg_analysis_lib import erp_alltrials

erp_alltrials(mode= "Audio", rejection_th =70)
erp_alltrials(mode= "Vibro", rejection_th =70)
erp_alltrials(mode= "Shape", rejection_th =70)

------------------------
PSD for all raws, left n right
----------------------